In [ ]:
!pip install zarr
!pip install -U xarray

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# %load_ext autoreload
# %autoreload 2

import xarray as xr
import zarr
import os
import pandas as pd
from functools import partial
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import datetime
import copy
import seaborn as sns
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import L1L2
from functools import partial

import sys
sys.path.append('amrakesh28')
sys.path.append('/content/drive/MyDrive/syde770')


from model_tester import *

In [ ]:
# Read and drop unnecessary variables
zarr_path = 'data_full.zarr' if 'google.colab' not in sys.modules else '/content/drive/MyDrive/syde770/data_full.zarr'
save_dir = 'model_data_fixed/' if 'google.colab' not in sys.modules else '/content/drive/MyDrive/syde770/model_data_fixed/'

ds = xr.open_zarr(zarr_path)
ds = xr.concat([
    ds.sel(time=slice(None, datetime.datetime(2011, 9, 30))), 
    ds.sel(time=slice(datetime.datetime(2012, 10, 1), None))
    ], dim='time')
ds = ds.drop(['Lambert_Azimuthal_Grid', 'status_flag', 'ceda_sic_bin', 'era5_sic', 'era5_sic_bin', 'total_standard_error'])
loss = masked_MSE

# Pre-process parameters
data_params_dict = {
    # ====== WEEKLY ======
    # --- 1 week ahead ---
    'Weekly_4T_0G': dict(
        weekly=True,
        num_timesteps=4,
        gap=0,
    ),
    'Weekly_8T_0G': dict(
        weekly=True,
        num_timesteps=4,
        gap=0,
    ),
    # --- 1 week ahead ---
    'Weekly_4T_2G': dict(
        weekly=True,
        num_timesteps=4,
        gap=1,
    ),
    'Weekly_8T_2G': dict(
        weekly=True,
        num_timesteps=4,
        gap=1,
    ),
    # --- 2 week ahead ---
    'Weekly_4T_2G': dict(
        weekly=True,
        num_timesteps=4,
        gap=2,
    ),
    'Weekly_8T_2G': dict(
        weekly=True,
        num_timesteps=4,
        gap=2,
    ),
    # ====== DAILY ======
    # --- 0 week ahead ---
    'Daily_3T_0G': dict(
        weekly=False,
        num_timesteps=3,
        gap=0,
    ),
    'Daily_7T_0G': dict(
        weekly=False,
        num_timesteps=7,
        gap=0,
    ),
    'Daily_14T_0G': dict(
        weekly=False,
        num_timesteps=14,
        gap=0,
    ),
    # --- 1 week ahead ---
    'Daily_3T_7G': dict(
        weekly=False,
        num_timesteps=3,
        gap=7,
    ),
    'Daily_7T_7G': dict(
        weekly=False,
        num_timesteps=7,
        gap=7,
    ),
    'Daily_14T_7G': dict(
        weekly=False,
        num_timesteps=14,
        gap=7,
    ),
    # --- 2 week ahead ---
    'Daily_3T_14G': dict(
        weekly=False,
        num_timesteps=3,
        gap=14,
    ),
    'Daily_7T_14G': dict(
        weekly=False,
        num_timesteps=7,
        gap=14,
    ),
    'Daily_14T_14G': dict(
        weekly=False,
        num_timesteps=14,
        gap=14,
    ),
}

# Model parameters
model_params_dict = {
    'Heavy': dict(
        num_convlstm=4,
        convlstm_filters = [128, 128, 64, 64],
        convlstm_kernels = [(7, 7), (5, 5), (3, 3), (1, 1)],
        convlstm_rec_dropout = 0.1,
        convlstm_dropout = 0.1,
        convlstm_kernal_reg = L1L2(0.001, 0.001),
        num_conv=3,
        conv_filters=[64, 32, 32],
        conv_kernels=[(5, 5), (3, 3), (1, 1)],
    ),
    'Medium': dict(
        num_convlstm=3,
        convlstm_filters = [64, 64, 64],
        convlstm_kernels = [(5, 5), (3, 3), (1, 1)],
        convlstm_rec_dropout = 0.1,
        convlstm_dropout = 0.1,
        convlstm_kernal_reg = L1L2(0.001, 0.001),
        num_conv=2,
        conv_filters=[32, 32],
        conv_kernels=[(3, 3), (1, 1)],
    ),
    'Light': dict(
        num_convlstm=2,
        convlstm_filters = [64, 64],
        convlstm_kernels = [(3, 3), (1, 1)],
        convlstm_rec_dropout = 0.1,
        convlstm_dropout = 0.1,
        convlstm_kernal_reg = L1L2(0.001, 0.001),
        num_conv=1,
        conv_filters=[32],
        conv_kernels=[(1, 1)],
    ),
    'Extra-light': dict(
        num_convlstm=1,
        convlstm_filters = [32],
        convlstm_kernels = [(3, 3)],
        convlstm_rec_dropout = 0.1,
        convlstm_dropout = 0.1,
        convlstm_kernal_reg = L1L2(0.001, 0.001),
        num_conv=1,
        conv_filters=[16],
        conv_kernels=[(1, 1)],
    ),
}

In [ ]:
save_dir = '/content/drive/MyDrive/syde770/save_dir_5/'
model_tester = ModelTester()

i = 0
num_tests = len(data_params_dict) * len(model_params_dict)

for data_params_name, data_params in data_params_dict.items():
    model_tester.preprocess_data(ds, **data_params)

    for model_params_name, model_params in model_params_dict.items():
        print(f'{i} / {num_tests}')
        i += 1

        model_name = data_params_name + '_' + model_params_name

        filename = save_dir + model_name
        if os.path.exists(filename + '.p'):
            continue

        model_tester.create_model(
            loss=loss(mask=np.expand_dims(~model_tester.nan_mask, [0, -1])), 
            **model_params
            )
        
        model_tester.train(verbose=1, epochs=500, batch_size=20)

        model_tester.save(model_name, save_dir)

/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


0 / 40
Epoch 1/500
8/8 [==============================] - 19s 625ms/step - loss: 19.1078 - val_loss: 11.8431 - lr: 0.0010
Epoch 2/500
8/8 [==============================] - 3s 440ms/step - loss: 8.8952 - val_loss: 5.7599 - lr: 0.0010
Epoch 3/500
8/8 [==============================] - 3s 440ms/step - loss: 5.1605 - val_loss: 4.0928 - lr: 0.0010
Epoch 4/500
8/8 [==============================] - 3s 441ms/step - loss: 3.3719 - val_loss: 2.3541 - lr: 0.0010
Epoch 5/500
8/8 [==============================] - 3s 440ms/step - loss: 2.1870 - val_loss: 1.6950 - lr: 0.0010
Epoch 6/500
8/8 [==============================] - 3s 440ms/step - loss: 1.6639 - val_loss: 1.3343 - lr: 0.0010
Epoch 7/500
8/8 [==============================] - 3s 441ms/step - loss: 1.3300 - val_loss: 1.0694 - lr: 0.0010
Epoch 8/500
8/8 [==============================] - 3s 441ms/step - loss: 1.1265 - val_loss: 0.9395 - lr: 0.0010
Epoch 9/500
8/8 [==============================] - 3s 440ms/step - loss: 0.9887 - val_loss: 0.

/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


4 / 40
Epoch 1/500
8/8 [==============================] - 17s 586ms/step - loss: 19.4361 - val_loss: 12.0533 - lr: 0.0010
Epoch 2/500
8/8 [==============================] - 3s 440ms/step - loss: 9.2122 - val_loss: 5.9577 - lr: 0.0010
Epoch 3/500
8/8 [==============================] - 3s 441ms/step - loss: 5.4673 - val_loss: 4.2855 - lr: 0.0010
Epoch 4/500
8/8 [==============================] - 3s 441ms/step - loss: 3.6697 - val_loss: 2.5385 - lr: 0.0010
Epoch 5/500
8/8 [==============================] - 3s 440ms/step - loss: 2.4750 - val_loss: 1.8720 - lr: 0.0010
Epoch 6/500
8/8 [==============================] - 3s 442ms/step - loss: 1.9453 - val_loss: 1.5067 - lr: 0.0010
Epoch 7/500
8/8 [==============================] - 3s 440ms/step - loss: 1.6042 - val_loss: 1.2362 - lr: 0.0010
Epoch 8/500
8/8 [==============================] - 3s 441ms/step - loss: 1.3934 - val_loss: 1.1015 - lr: 0.0010
Epoch 9/500
8/8 [==============================] - 3s 440ms/step - loss: 1.2492 - val_loss: 0.

/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


8 / 40
Epoch 1/500
8/8 [==============================] - 18s 647ms/step - loss: 19.2682 - val_loss: 11.8836 - lr: 0.0010
Epoch 2/500
8/8 [==============================] - 3s 438ms/step - loss: 9.0095 - val_loss: 5.7975 - lr: 0.0010
Epoch 3/500
8/8 [==============================] - 3s 437ms/step - loss: 5.2463 - val_loss: 4.1321 - lr: 0.0010
Epoch 4/500
8/8 [==============================] - 3s 437ms/step - loss: 3.4601 - val_loss: 2.3922 - lr: 0.0010
Epoch 5/500
8/8 [==============================] - 3s 437ms/step - loss: 2.2682 - val_loss: 1.7324 - lr: 0.0010
Epoch 6/500
8/8 [==============================] - 3s 437ms/step - loss: 1.7418 - val_loss: 1.3715 - lr: 0.0010
Epoch 7/500
8/8 [==============================] - 3s 437ms/step - loss: 1.4074 - val_loss: 1.1062 - lr: 0.0010
Epoch 8/500
8/8 [==============================] - 3s 437ms/step - loss: 1.2024 - val_loss: 0.9769 - lr: 0.0010
Epoch 9/500
8/8 [==============================] - 3s 438ms/step - loss: 1.0637 - val_loss: 0.

/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


12 / 40
Epoch 1/500
8/8 [==============================] - 17s 580ms/step - loss: 19.4317 - val_loss: 11.9848 - lr: 0.0010
Epoch 2/500
8/8 [==============================] - 3s 438ms/step - loss: 9.1644 - val_loss: 5.8879 - lr: 0.0010
Epoch 3/500
8/8 [==============================] - 3s 438ms/step - loss: 5.3908 - val_loss: 4.2150 - lr: 0.0010
Epoch 4/500
8/8 [==============================] - 3s 437ms/step - loss: 3.5938 - val_loss: 2.4679 - lr: 0.0010
Epoch 5/500
8/8 [==============================] - 3s 438ms/step - loss: 2.3932 - val_loss: 1.8020 - lr: 0.0010
Epoch 6/500
8/8 [==============================] - 3s 438ms/step - loss: 1.8627 - val_loss: 1.4390 - lr: 0.0010
Epoch 7/500
8/8 [==============================] - 3s 437ms/step - loss: 1.5235 - val_loss: 1.1698 - lr: 0.0010
Epoch 8/500
8/8 [==============================] - 3s 438ms/step - loss: 1.3142 - val_loss: 1.0379 - lr: 0.0010
Epoch 9/500
8/8 [==============================] - 3s 438ms/step - loss: 1.1717 - val_loss: 0

In [ ]:
import glob
model_names = [os.path.basename(fn.split('.')[0]) for fn in glob.glob(save_dir + '*.p') if not fn.endswith('model.p')]

model_tester = ModelTester()

results = []
for model_name in model_names:
    print(model_name)
    model_tester.load(model_name, save_dir, load_model=True)
    results.append(model_tester.df.loc['Test RMSE'])
    
df = pd.concat(results, axis=1)
df.columns = model_names
df = df.T


df = df.join(pd.DataFrame([n.split('_') for n in df.index], columns=['Scale', 'Timesteps', 'Gap', 'Model'], index=df.index))

df['Timesteps'] = [int(s[:-1]) for s in df.Timesteps]
df['Gap'] = [int(s[:-1]) for s in df.Gap]
df['Model'] = pd.Categorical(df['Model'], ['Extra-light', 'Light', 'Medium', 'Heavy', 'Extra-heavy'])

df = df.sort_values(['Timesteps', 'Gap', 'Model'])

df = df[df.Model != 'Extra-heavy']  # Remove extra heavy models. Not good and take too long to run.

# Threshold for RMSE over baseline to be considered not converged
thresh = 0.75
# df = df[(df.NN - df[['Persistence', 'Climatology']].min(axis=1)) < thresh]

df

Weekly_4T_0G_Heavy


,NN,Persistence,Climatology,Scale,Timesteps,Gap,Model
Weekly_4T_0G_Heavy,3.432813,4.466472,3.561855,Weekly,4,0,Heavy
